In [1]:
import caffe
import tensorflow as tf
import numpy as np
import cv2
import os
import mxnet as mx
from mxnet import image
import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

/home/dluser1/miniconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
num_points = 8

model_stage1 = '../models/FLD_full/stage1.prototxt'
assert(os.path.exists(model_stage1))
weights_stage1 = '../models/FLD_full/FLD_full_models/stage1.caffemodel'
assert(os.path.exists(weights_stage1))

model_stage2 = '../models/FLD_full/cascade.prototxt'
assert(os.path.exists(model_stage2))
weights_stage2 = '../models/FLD_full/FLD_full_models/stage2.caffemodel'
assert(os.path.exists(weights_stage2))

model_stage3 = '../models/FLD_full/cascade.prototxt'
assert(os.path.exists(model_stage3))
weights_stage3_easy = '../models/FLD_full/FLD_full_models/stage3_easy.caffemodel'
assert(os.path.exists(weights_stage3_easy))
weights_stage3_hard = '../models/FLD_full/FLD_full_models/stage3_hard.caffemodel'
assert(os.path.exists(weights_stage3_hard))

In [3]:
# caffe prepare
caffe.set_mode_gpu()
caffe.set_device(0)

In [4]:
# create net and load weights
net_stage1 = caffe.Net(model_stage1, weights_stage1, caffe.TEST)
# display([(k, v.data.shape) for k, v in net_stage1.blobs.items()])

net_stage2 = caffe.Net(model_stage2, weights_stage2, caffe.TEST)
# display([(k, v.data.shape) for k, v in net_stage2.blobs.items()])

net_stage3_easy = caffe.Net(model_stage3, weights_stage3_easy, caffe.TEST)
net_stage3_hard = caffe.Net(model_stage3, weights_stage3_hard, caffe.TEST)
pipeline = {
    'num_points': num_points,
    'net_stage1': net_stage1,
    'net_stage2': net_stage2,
    'net_stage3_easy': net_stage3_easy,
    'net_stage3_hard': net_stage3_hard
}

In [5]:
def print_layer_info(net):
    """Print the names and the output shapes of the layers"""
    for layer_name, blob in net.blobs.items():
        print('The shape of layer {} is {}'.format(layer_name, blob.data.shape[1:]))

In [6]:
print_layer_info(net_stage1)

The shape of layer data is (3, 224, 224)
The shape of layer conv1_1 is (64, 224, 224)
The shape of layer conv1_2 is (64, 224, 224)
The shape of layer pool1 is (64, 112, 112)
The shape of layer conv2_1 is (128, 112, 112)
The shape of layer conv2_2 is (128, 112, 112)
The shape of layer pool2 is (128, 56, 56)
The shape of layer conv3_1 is (256, 56, 56)
The shape of layer conv3_2 is (256, 56, 56)
The shape of layer conv3_3 is (256, 56, 56)
The shape of layer pool3 is (256, 28, 28)
The shape of layer conv4_1 is (512, 28, 28)
The shape of layer conv4_2 is (512, 28, 28)
The shape of layer conv4_3 is (512, 28, 28)
The shape of layer pool4 is (512, 14, 14)
The shape of layer conv5_1 is (512, 14, 14)
The shape of layer conv5_2 is (512, 14, 14)
The shape of layer conv5_3 is (512, 14, 14)
The shape of layer pool5 is (512, 7, 7)
The shape of layer fc6 is (4096,)
The shape of layer fc7 is (4096,)
The shape of layer fc7_drop7_0_split_0 is (4096,)
The shape of layer fc7_drop7_0_split_1 is (4096,)
The 

In [10]:
# print_layer_info(net_stage2)

In [8]:
# print_layer_info(net_stage3_easy)
# print_layer_info(net_stage3_hard)

In [18]:
from keras.models import Model
from keras.layers import Input, Flatten, Dense, Conv2D, MaxPooling2D, Concatenate, Convolution2D, ZeroPadding2D
from keras import backend as K

In [19]:
img_input = Input(shape=(3, 224, 224), name='data')

# stage 1 model
x = Convolution2D(64, (3, 3), activation='relu', name='conv1_1')(img_input)
x = ZeroPadding2D((1, 1))(x)
x = Convolution2D(64, (3, 3), activation='relu', name='conv1_2')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='pool1')(x)
x = Convolution2D(128, (3, 3), activation='relu', name='conv2_1')(x)
x = Convolution2D(128, (3, 3), activation='relu', name='conv2_2')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='pool2')(x)
x = Convolution2D(256, (3, 3), activation='relu', name='conv3_1')(x)
x = Convolution2D(256, (3, 3), activation='relu', name='conv3_2')(x)
x = Convolution2D(256, (3, 3), activation='relu', name='conv3_3')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='pool3')(x)
x = Convolution2D(512, (3, 3), activation='relu', name='conv4_1')(x)
x = Convolution2D(512, (3, 3), activation='relu', name='conv4_2')(x)
x = Convolution2D(512, (3, 3), activation='relu', name='conv4_3')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='pool4')(x)
x = Convolution2D(512, (3, 3), activation='relu', name='conv5_1')(x)
x = Convolution2D(512, (3, 3), activation='relu', name='conv5_2')(x)
x = Convolution2D(512, (3, 3), activation='relu', name='conv5_3')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='pool5')(x)
x = Dense(4096, activation='relu', name='fc6')(x)
x = Dropout(0.5, name='drop6')(x)
x = Dense(4096, activation='relu', name='fc7')(x)
x = Dropout(0.5, name='drop7')(x)
# concat layer
y0 = Dense(20, activation='softmax', name='fc8_softlabel')(x)
x1 = Dense(16, name='fc8_landmarks')(x)
x2 = Dense(3, name='fc8_visibility_1')(x)
x3 = Dense(3, name='fc8_visibility_2')(x)
x4 = Dense(3, name='fc8_visibility_3')(x)
x5 = Dense(3, name='fc8_visibility_4')(x)
x6 = Dense(3, name='fc8_visibility_5')(x)
x7 = Dense(3, name='fc8_visibility_6')(x)
x8 = Dense(3, name='fc8_visibility_7')(x)
x9 = Dense(3, name='fc8_visibility_8')(x)
# last layer
y1 = Concatenate([x1, x2, x3, x4, x5, x6, x7, x8, x9]) 

/home/dluser1/miniconda3/lib/python3.6/site-packages/tensorflow/python/framework/tensor_util.py:560: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  return np.fromstring(tensor.tensor_content, dtype=dtype).reshape(shape)


ValueError: Negative dimension size caused by subtracting 2 from 1 for 'pool1/MaxPool' (op: 'MaxPool') with input shapes: [?,1,222,64].